# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [29]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [30]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [31]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [32]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [33]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print("Error: could not connect to the local Cassandra cluster")
    print(e)

#### Create Keyspace

In [34]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifydb 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

except Exception as e:
    print("Error: coult not create sparkifydb keyspace")
    print(e)

#### Set Keyspace

In [35]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print("Error: could not connect to sparkifydb keyspace")
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

For this query, it is needed that a table store at least artist name, song title, song length, as these are the information needed to be returned by the query, and session id and item in session, since the query will be filtered by these items.

Since filtering will be done based on session id and item in session, they need to be part of the partition key

The final table has
- `session_id` as an integer
- `item_in_session` as an integer
- `artist` as text, since it's a name
- `song_title` as text, since it's also a name
- `song_length` as float


### Defining the create, drop, insert and select queries

In [36]:
session_history_table_create = ("""
    CREATE TABLE IF NOT EXISTS session_history(
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        song_length float,
        PRIMARY KEY(session_id, item_in_session)
    )
""")

session_history_table_drop = "DROP TABLE IF EXISTS session_history"

session_history_table_insert = ("""
    INSERT INTO session_history(
        session_id,
        item_in_session,
        artist,
        song_title,
        song_length
    ) VALUES (%s, %s, %s, %s, %s)
""")

session_history_table_select = "SELECT artist, song_title, song_length FROM session_history WHERE session_id = ? AND item_in_session = ?"

### Creating session_history table

In [37]:
try:
    session.execute(session_history_table_create)
except Exception as e:
    print("Error: could not create session_history table")
    print(e)

### Adding records to table

In [38]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session_id = int(line[8])
        item_in_session = int(line[3])
        artist = line[0]
        song_title = line[9]
        song_length = float(line[5])
        try:
            session.execute(session_history_table_insert, (session_id, item_in_session, artist, song_title, song_length))
        except Exception as e:
            print("Error: could not insert into session_history table")
            print(e)

### Do a SELECT to verify that the data have been inserted into each table

In [39]:
try:
    session_history_table_select_stmt = session.prepare(session_history_table_select)
    rows = session.execute(session_history_table_select_stmt, [338, 4])
except Exception as e:
    print("Error: could not execute select on session_history table")
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For this query, it is needed that a table store at least artist name, song title and user (first and last name), as these are the information needed to be returned by the query, and user id, session id and item in session, since the query will be filtered or ordered by these items.

Since filtering will be done based on user id and session id, they need to be part of the partition key
Since ordering will be done based on item in session, it will be a clustering key

The final table has
- `user_id` as an integer
- `session_id` as an integer
- `item_in_session` as an integer
- `artist` as text, since it's a name
- `song_title` as text, since it's also a name
- `user` as text. `user` field contain first and last name combined

### Defining the create, drop, insert and select queries

In [40]:
user_history_table_create = ("""
    CREATE TABLE IF NOT EXISTS user_history(
        user_id int,
        session_id int,
        item_in_session int,
        artist text,
        song_title text,
        user text,
        PRIMARY KEY((user_id, session_id), item_in_session)
    ) WITH CLUSTERING ORDER BY (item_in_session ASC)
""")

user_history_table_drop = "DROP TABLE IF EXISTS user_history"

user_history_table_insert = ("""
    INSERT INTO user_history(
        user_id,
        session_id,
        item_in_session,
        artist,
        song_title,
        user
    ) VALUES (%s, %s, %s, %s, %s, %s)
""")

user_history_table_select = "SELECT artist, song_title, user FROM user_history WHERE user_id = ? AND session_id = ?"                    

### Creating user_history table

In [41]:
try:
    session.execute(user_history_table_create)
except Exception as e:
    print("Error: could not create user_history table")
    print(e)

### Adding records to table

In [42]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user_id = int(line[10])
        session_id = int(line[8])
        item_in_session = int(line[3])
        artist = line[0]
        song_title = line[9]
        user = "{} {}".format(line[1], line[4])
        try:
            session.execute(user_history_table_insert, (user_id, session_id, item_in_session, artist, song_title, user))
        except Exception as e:
            print("Error: could not insert into user_history table")
            print(e)

### Do a SELECT to verify that the data have been inserted into each table

In [43]:
try:
    user_history_table_select_stmt = session.prepare(user_history_table_select)
    rows = session.execute(user_history_table_select_stmt, [10, 182])
except Exception as e:
    print("Error: could not execute select on user_history table")
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For this query, it is needed that a table store at least the user (first and last name), as this information need to be returned by the query, and song_title, since the query will be filtered by this.
However, the combination of user name and song is not a good primary key, because there could be people with the same name. We add to the table a user id, because this can be used along with song to uniquely identify songs listened by users.

Since filtering will be done based on song_title, it needs to be part of the partition key.
The field `user_id` is also part of the partition key for reasons commented above.

The final table has
- `song_title` as text, since it's a name
- `user_id` as an integer
- `user` as text. `user` field contain first and last name combined

### Defining the create, drop, insert and select queries

In [44]:
song_history_table_create = ("""
    CREATE TABLE IF NOT EXISTS song_history(
        song_title text,
        user_id int,
        user text,
        PRIMARY KEY(song_title, user_id)
    )
""")

song_history_table_drop = "DROP TABLE IF EXISTS song_history"

song_history_table_insert = ("""
    INSERT INTO song_history(
        song_title,
        user_id,
        user
    ) VALUES (%s, %s, %s)
""")

song_history_table_select = "SELECT user FROM song_history WHERE song_title = ?"

### Creating song_history table

In [45]:
try:
    session.execute(song_history_table_create)
except Exception as e:
    print("Error: could not create song_history table")
    print(e)

### Adding records to table

In [46]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        song_title = line[9]
        user_id = int(line[10])
        user = "{} {}".format(line[1], line[4])
        try:
            session.execute(song_history_table_insert, (song_title, user_id, user))
        except Exception as e:
            print("Error: could not insert into song_history table")
            print(e)

### Do a SELECT to verify that the data have been inserted into each table

In [47]:
try:
    song_history_table_select_stmt = session.prepare(song_history_table_select)
    rows = session.execute(song_history_table_select_stmt, ["All Hands Against His Own"])
except Exception as e:
    print("Error: could not execute select on song_history table")
    print(e)
    
for row in rows:
    print (row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [48]:
queries = [
    session_history_table_drop,
    user_history_table_drop,
    song_history_table_drop
]

for query in queries:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()